# Dual-Collaborative Filtering Autoencoder Metric Network

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import math
import os


def sparseEmbed(df, name, num, colIdx):
    embedName = [ name+"_"+str(i) for i in range(num)] 
    Emptydf = pd.DataFrame()
    Emptydf[embedName] = df[name].str.split('|',expand=True)
    values = np.unique(Emptydf[embedName].values)
    
    dic = {}
    a = 0
    for i in values:
        dic[i] = a
        a += 1
    dic.pop('nan', None)
    
    
    appendValue = np.zeros([Emptydf.values.shape[0], len(values)])
    for i in range(Emptydf.values.shape[0]):
        for j in range(num):
            key = Emptydf.values[i][j]
            if key in dic:
                appendValue[i][dic[key]] = 1
    
    for i in range(appendValue.shape[1], 0, -1):
        df.insert(colIdx, name+"_"+str(i-1), appendValue[:, i-1])
    
    del df[name]
    return df

def toDummy(df, name, colIdx):
    num = len(np.unique(df[name].values.astype(str)))-1
    embedName = [ name+"_"+str(i) for i in range(num)]  # don't need nan value
        
    dic = {}
    a = 0
    for i in range(num+1):
        dic[i] = a
        a += 1
    dic.pop('nan', None)
        
    appendValue = np.zeros([df[name].size, a])
    for i in range(df[name].size):
        key = df[name].values[i]
        if key in dic:
            appendValue[i][dic[key]] = 1
    
    for i in range(appendValue.shape[1], 0, -1):
        df.insert(colIdx, name+"_"+str(i-1), appendValue[:, i-1])
    
    del df[name]
    return df

def genderDummy(df, name, colIdx):
    pool = set()
    num = len(np.unique(df[name].values))-1
    for i in df[name].values:
        pool.add(str(i))
    num = len(list(pool))-1
    embedName = [ name+"_"+str(i) for i in range(num)]  # don't need nan value
        
    dic = {}
    a = 0
    for i in range(num+1):
        dic[i] = a
        a += 1
    dic.pop('nan', None)
        
    appendValue = np.zeros([df[name].size, a])
    for i in range(df[name].size):
        key = df[name].values[i]
        if key in dic:
            appendValue[i][dic[key]] = 1
    
    for i in range(appendValue.shape[1], 0, -1):
        df.insert(colIdx, name+"_"+str(i-1), appendValue[:, i-1])
    
    del df[name]
    return df

# Importing data and transforming to categorical binary input data form

In [2]:
head = ["user_age", "user_gender", "user_7_hero", "user_30_hero", "user_7_keyword", "user_7_author", "item_rate", "item_keyword", "item_author", "item_avgTime", "item_numReader", "item_numTime", "label"]
raw = pd.read_csv("./thing.txt", names=head, sep=",", index_col = False)

colIdx = raw.columns.values.tolist().index("user_gender")
raw = genderDummy(raw, "user_gender", colIdx)
colIdx = raw.columns.values.tolist().index("item_keyword")
raw = toDummy(raw, "item_keyword", colIdx)

numDic = {"user_gender": 1, "user_7_hero": 5, "user_30_hero": 5, "user_7_keyword": 3, "user_7_author": 3, "item_keyword": 1, "item_author": 3}
for i in ["user_7_hero", "user_30_hero", "user_7_keyword", "user_7_author", "item_author"]:
    colIdx = raw.columns.values.tolist().index(i)
    raw = sparseEmbed(raw, i, numDic[i], colIdx)
    print("finished with", i)

# normalize numerical features into interval [0, 1]
for i in ["user_age", "item_rate", "item_avgTime", "item_numReader", "item_numTime"]:
    r = raw[i].values.astype(float)
    min_max_scaler = MinMaxScaler()
    x_scaled = min_max_scaler.fit_transform(r.reshape(-1,1))
    raw_normalized = pd.DataFrame(x_scaled)
    raw[i] = raw_normalized

raw = raw.sample(200000)
    
raw.head()

finished with user_7_hero
finished with user_30_hero
finished with user_7_keyword
finished with user_7_author
finished with item_author


user_age  user_gender_0  user_gender_1  user_gender_2  user_gender_3  \
322051  0.293333            0.0            1.0            0.0            0.0   
309142  0.213333            0.0            1.0            0.0            0.0   
39702   0.373333            0.0            1.0            0.0            0.0   
232882  0.293333            0.0            1.0            0.0            0.0   
30629   0.280000            0.0            1.0            0.0            0.0   

        user_7_hero_0  user_7_hero_1  user_7_hero_2  user_7_hero_3  \
322051            0.0            0.0            0.0            0.0   
309142            0.0            0.0            0.0            0.0   
39702             0.0            0.0            0.0            0.0   
232882            0.0            0.0            0.0            0.0   
30629             0.0            0.0            0.0            0.0   

        user_7_hero_4  ...  item_author_519  item_author_520  item_author_521  \
322051            0.0  ...              0.0              0.0              0.0   
309142            1.0  ...              0.0              0.0              0.0   
39702             0.0  ...              0.0              0.0              0.0   
232882            0.0  ...              0.0              0.0              0.0   
30629             0.0  ...              0.0              0.0              0.0   

        item_author_522  item_author_523  item_author_524  item_avgTime  \
322051              0.0              0.0              0.0      0.020649   
309142              0.0              0.0              0.0      0.045477   
39702               0.0              0.0              0.0      0.022700   
232882              0.0              0.0              0.0      0.055037   
30629               0.0              0.0              0.0      0.049202   

        item_numReader  item_numTime  label  
322051        0.038687      0.005559    0.0  
309142        0.062402      0.019748    0.0  
39702         0.068731      0.010857    1.0  
232882        0.250331      0.095872    0.0  
30629         0.628562      0.215205    0.0  

[5 rows x 2119 columns]

In [3]:

data = raw.sample(5000)

# Splitting dataframe into train, validation, and testing
dataY = data['label'].values
dataX = data.drop(columns = 'label').values


X, Xtest, Y, Ytest = train_test_split(dataX, dataY, test_size = 0.2, random_state = 42)
Xtrain, Xval, Ytrain, Yval = train_test_split(X, Y, test_size = 0.25, random_state = 42)

print("training data size: {} | validate data size: {} | testing data size: {}".format(str(Xtrain.shape), str(Xval.shape), str(Xtest.shape)))


# Training Parameters
learning_rate = 0.000001
batch_size = 256
epochs = 800


# Network Parameters
num_input = Xtrain.shape[1]
num_input_p = data.columns.values.tolist().index("item_rate") # number of all user input columns, the last column ends before the start of "item_rate" column
num_input_g = data.columns.values.shape[0] - num_input_p - 1   # number of all item input columns, = all column -user -label

a = 1
num_encode_1 = int(256 *a)
num_encode_2 = int(128 *a)
num_encode_3 = int(64 *a)


num_neck = 5

num_decode_1 = num_encode_3
num_decode_2 = num_encode_2
num_decode_3 = num_encode_1

num_output_to_p = num_input_p
num_output_to_g = num_input_g

#del raw


# balance weight coefficient [0,1], the bigger the mode focused onto neck distance
alpha = 0.5

# regularization
beta = 0.0005

# collaborative autoencoder input tensor1
X = tf.placeholder("float", [None, num_input])
label = tf.placeholder("float", [None])
threshold = tf.Variable(0.0)

weights = {
    'encoder_ph1': tf.Variable(tf.random_normal([num_input_p , num_encode_1])),
    'encoder_gh1': tf.Variable(tf.random_normal([num_input_g , num_encode_1])),
    'encoder_ph2': tf.Variable(tf.random_normal([num_encode_1 , num_encode_2])),
    'encoder_gh2': tf.Variable(tf.random_normal([num_encode_1 , num_encode_2])),
    'encoder_ph3': tf.Variable(tf.random_normal([num_encode_2 , num_encode_3])),
    'encoder_gh3': tf.Variable(tf.random_normal([num_encode_2 , num_encode_3])),    


    'encoder_pneck': tf.Variable(tf.random_normal([num_encode_3 , num_neck])), ## METRIC SPACE OF PERSON
    'encoder_gneck': tf.Variable(tf.random_normal([num_encode_3 , num_neck])), ## METRIC SPACE OF GOODS


    'decoder_ph1': tf.Variable(tf.random_normal([num_neck , num_decode_1])),
    'decoder_gh1': tf.Variable(tf.random_normal([num_neck , num_decode_1])),
    'decoder_ph2': tf.Variable(tf.random_normal([num_decode_1 , num_decode_2])),
    'decoder_gh2': tf.Variable(tf.random_normal([num_decode_1 , num_decode_2])),
    'decoder_ph3': tf.Variable(tf.random_normal([num_decode_2 , num_decode_3])),
    'decoder_gh3': tf.Variable(tf.random_normal([num_decode_2 , num_decode_3])),    


    'decoder_p_to_p_out': tf.Variable(tf.random_normal([num_decode_3 , num_output_to_p])),
    'decoder_g_to_g_out': tf.Variable(tf.random_normal([num_decode_3 , num_output_to_g]))
}

biases = {  
    'encoder_bph1': tf.Variable(tf.random_normal([num_encode_1])),
    'encoder_bgh1': tf.Variable(tf.random_normal([num_encode_1])),
    'encoder_bph2': tf.Variable(tf.random_normal([num_encode_2])),
    'encoder_bgh2': tf.Variable(tf.random_normal([num_encode_2])),
    'encoder_bph3': tf.Variable(tf.random_normal([num_encode_3])),
    'encoder_bgh3': tf.Variable(tf.random_normal([num_encode_3])),


    'encoder_bpneck': tf.Variable(tf.random_normal([num_neck])), ## METRIC SPACE OF PERSON
    'encoder_bgneck': tf.Variable(tf.random_normal([num_neck])), ## METRIC SPACE OF GOODS


    'decoder_bph1': tf.Variable(tf.random_normal([num_decode_1])),
    'decoder_bgh1': tf.Variable(tf.random_normal([num_decode_1])),
    'decoder_bph2': tf.Variable(tf.random_normal([num_decode_2])),
    'decoder_bgh2': tf.Variable(tf.random_normal([num_decode_2])),
    'decoder_bph3': tf.Variable(tf.random_normal([num_decode_3])),
    'decoder_bgh3': tf.Variable(tf.random_normal([num_decode_3])),    


    'decoder_b_p_to_p_out': tf.Variable(tf.random_normal([num_output_to_p])),
    'decoder_b_g_to_g_out': tf.Variable(tf.random_normal([num_output_to_g]))
}

# Building the encoder
def encoder(x):

    ## Person encoder:
    layer_p_1 = tf.nn.relu(tf.add(tf.matmul(x[:, :1376], weights['encoder_ph1']), biases['encoder_bph1']))  ## HARD CODING: 1375 is the ending index of person feature; 1376 the starting index of goods feature
    layer_p_2 = tf.nn.relu(tf.add(tf.matmul(layer_p_1, weights['encoder_ph2']), biases['encoder_bph2']))
    layer_p_3 = tf.nn.relu(tf.add(tf.matmul(layer_p_2, weights['encoder_ph3']), biases['encoder_bph3']))

    layer_p_neck = tf.nn.sigmoid(tf.add(tf.matmul(layer_p_3, weights['encoder_pneck']), biases['encoder_bpneck']))

    ## Good encoder
    layer_g_1 = tf.nn.relu(tf.add(tf.matmul(x[:, 1376:], weights['encoder_gh1']), biases['encoder_bgh1']))  ## HARD CODING: 1375 is the ending index of person feature; 1376 the starting index of goods feature
    layer_g_2 = tf.nn.relu(tf.add(tf.matmul(layer_g_1, weights['encoder_gh2']), biases['encoder_bgh2']))
    layer_g_3 = tf.nn.relu(tf.add(tf.matmul(layer_g_2, weights['encoder_gh3']), biases['encoder_bgh3']))

    layer_g_neck = tf.nn.sigmoid(tf.add(tf.matmul(layer_g_3, weights['encoder_gneck']), biases['encoder_bgneck']))


    return layer_p_neck, layer_g_neck


# Building the decoder
def decoder(p_neck, g_neck):

    ## Good to Person decoder
    layer_p_1 = tf.nn.relu(tf.add(tf.matmul(p_neck, weights['decoder_ph1']), biases['decoder_bph1']))
    layer_p_2 = tf.nn.relu(tf.add(tf.matmul(layer_p_1, weights['decoder_ph2']), biases['decoder_bph2']))
    layer_p_3 = tf.nn.relu(tf.add(tf.matmul(layer_p_2, weights['decoder_ph3']), biases['decoder_bph3']))

    layer_p_to_p_out = tf.nn.sigmoid(tf.add(tf.matmul(layer_p_3, weights['decoder_p_to_p_out']), biases['decoder_b_p_to_p_out']))



    ## Person to Good decoder
    layer_g_1 = tf.nn.relu(tf.add(tf.matmul(g_neck, weights['decoder_gh1']), biases['decoder_bgh1']))
    layer_g_2 = tf.nn.relu(tf.add(tf.matmul(layer_g_1, weights['decoder_gh2']), biases['decoder_bgh2']))
    layer_g_3 = tf.nn.relu(tf.add(tf.matmul(layer_g_2, weights['decoder_gh3']), biases['decoder_bgh3']))

    layer_g_to_g_out = tf.nn.sigmoid(tf.add(tf.matmul(layer_g_3, weights['decoder_g_to_g_out']), biases['decoder_b_g_to_g_out']))

    result = tf.concat([layer_p_to_p_out, layer_g_to_g_out], axis = 1)

    return result

# Construct model

encoder_p, encoder_g = encoder(X)
decoder_out = decoder(encoder_p, encoder_g)

# Prediction
y_pred = decoder_out
# Targets (Labels) are the input data.
y_true = X

def getl2loss(dic):
    l2 = 0
    for i in dic.keys():
        l2 += tf.nn.l2_loss(dic[i])
    return l2

sign = 2*label-1


# calculate l2 distance
neck_distance_l2 = tf.reshape(tf.norm(encoder_p-encoder_g, axis = 1), [-1])
signed_distance_l2 = tf.multiply(neck_distance_l2, sign)



# calculate l infinity distance
neck_distance_linf = tf.reshape(tf.norm(encoder_p - encoder_g, axis=1, ord = np.infty), [-1])
signed_distance_linf = tf.multiply(neck_distance_linf, sign)




# 4 different losses
loss_neck_distance = tf.reduce_mean(tf.maximum(0.0, 0.6*threshold+tf.multiply(sign, signed_distance_l2-threshold)))



#signed_centered_distance = -(neck_distance - tf.reduce_mean(neck_distance))
# loss_neck_distance = tf.losses.hinge_loss(label, tf.multiply(sign, signed_distance_linf-threshold))
loss_pred_distance = tf.reduce_mean(tf.pow(y_true - y_pred, 2))
loss_weights = tf.reduce_sum(getl2loss(weights))
loss_bias = tf.reduce_sum(getl2loss(biases))

loss = alpha * loss_neck_distance + (1-alpha) * loss_pred_distance + beta * (loss_weights + loss_bias)


# Calculating AUC: 
# NOTE: because threshold is always half of maximum distance, 
# 2*threshold is maximum distance, normalize to scale of 1
# then use 1 to minus will yield a inversion that matches distance proximity property




auc, _ = tf.metrics.auc(label, tf.cast(tf.less_equal(neck_distance_linf, threshold,), tf.int32  ) )


# Define Optimizer
p_var_list = [weights["encoder_ph1"], weights["encoder_ph2"], weights["encoder_ph3"], weights["encoder_pneck"], 
              weights["decoder_ph1"], weights["decoder_ph2"], weights["decoder_ph3"], weights["decoder_p_to_p_out"],
              biases["encoder_bph1"], biases["encoder_bph2"], biases["encoder_bph3"], biases["encoder_bpneck"], 
              biases["decoder_bph1"], biases["decoder_bph2"], biases["decoder_bph3"], biases["decoder_b_p_to_p_out"]]

g_var_list = [weights["encoder_gh1"], weights["encoder_gh2"], weights["encoder_gh3"], weights["encoder_gneck"], 
              weights["decoder_gh1"], weights["decoder_gh2"], weights["decoder_gh3"], weights["decoder_g_to_g_out"],
              biases["encoder_bgh1"], biases["encoder_bgh2"], biases["encoder_bgh3"], biases["encoder_bgneck"], 
              biases["decoder_bgh1"], biases["decoder_bgh2"], biases["decoder_bgh3"], biases["decoder_b_g_to_g_out"]]

optimizer_p = tf.train.RMXPropOptimizer(learning_rate).minimize(loss, var_list = p_var_list)
optimizer_g = tf.train.RMXPropOptimizer(learning_rate).minimize(loss, var_list = g_var_list)

# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()
local_init = tf.local_variables_initializer()


# Start Training
# Start a new TF session
with tf.Session() as sess:


    # Run the initializer
    sess.run(init)
    sess.run(local_init)


    num_train_batches = int(Xtrain.shape[0] / batch_size)
    Xtrain = np.array_split(Xtrain, num_train_batches)
    Ytrain = np.array_split(Ytrain, num_train_batches)

    for i in range(len(Ytrain)):
        Ytrain[i] = np.reshape(Ytrain[i], [-1,1])
    Yval = np.reshape(Yval, [-1,1])



    # Training with validating
    true_false_break = math.sqrt(num_neck)/2

    for i in range(epochs):

        avg_train_cost = 0
        avg_train_neck_cost = 0
        avg_train_pred_cost = 0
        avg_train_auc =0
        for batch in range(len(Xtrain)):

            # optimize the person side
            _, l, neck_loss, pred_loss, neck_dis_train, train_auc = sess.run([optimizer_p, loss, loss_neck_distance, loss_pred_distance, neck_distance_l2, auc],  
                                                        #{optimizer:_, loss:l, loss_neck_distance:neck_loss, loss_pred_distance:pred_loss, neck_distance_l2:neck_dis}
                                                        feed_dict={X: Xtrain[batch], label: np.reshape(Ytrain[batch], [-1]), threshold: true_false_break})
           # optimize the goods side
            _, l, neck_loss, pred_loss, neck_dis_train, train_auc = sess.run([optimizer_g, loss, loss_neck_distance, loss_pred_distance, neck_distance_l2, auc],  
                                                        #{optimizer:_, loss:l, loss_neck_distance:neck_loss, loss_pred_distance:pred_loss, neck_distance_l2:neck_dis}
                                                        feed_dict={X: Xtrain[batch], label: np.reshape(Ytrain[batch], [-1]), threshold: true_false_break})

            avg_train_cost += l
            avg_train_neck_cost += neck_loss
            avg_train_pred_cost += pred_loss
            avg_train_auc += train_auc

            
        avg_train_cost /= num_train_batches
        avg_train_neck_cost /= num_train_batches
        avg_train_pred_cost /= num_train_batches
        avg_train_auc /= num_train_batches 
            

        # Validate once an epoch ends
        val_cost, val_neck_cost, val_pred_loss, neck_dis_val, val_auc = sess.run([loss, loss_neck_distance, loss_pred_distance, neck_distance_l2, auc],  
                                                        #{optimizer:_, loss:l, loss_neck_distance:neck_loss, loss_pred_distance:pred_loss, neck_distance_l2:neck_dis}
                                                        feed_dict={X: Xval, label: np.reshape(Yval, [-1]), threshold: true_false_break})

        if i % 50 == 0:
            # look at the training metric space
            signed_train = np.multiply(neck_dis_train, 2*Ytrain[batch].flatten()-1)
            pos = []
            neg = []
            for dist in signed_train:
                if dist >= 0:
                    pos.append(dist)
                else:
                    neg.append(dist)


            pos = np.array(pos)
            neg = np.array(neg)

            a = np.mean(pos)
            b = np.mean(neg)


            # look at the validation metric space
            signed_val = np.multiply(neck_dis_val.flatten(), 2*Yval.flatten()-1)
            pos_val = []
            neg_val = []
            for dist_val in signed_val:
                if dist_val >= 0:
                    pos_val.append(dist_val)
                else:
                    neg_val.append(dist_val)

            pos_val = np.array(pos_val)
            neg_val = np.array(neg_val)

            a_val = np.mean(pos_val)
            b_val = np.mean(neg_val)


            if i >= 2000:
                plt.figure(figsize=(12, 4))
                sns.set(color_codes=True)
                sns.distplot(pos, bins=20, kde = False, color="r", label="red: pos dis")
                sns.distplot(-neg, bins=20, kde = False , color="b", label="blue: neg dis")
                plt.legend()
                plt.xlim(0, 1)
                plt.show()

                plt.figure(figsize=(12, 4))
                sns.set(color_codes=True)
                sns.distplot(pos_val, bins=20, kde = False, color="r", label="red: pos dis")
                sns.distplot(-neg_val, bins=20, kde = False , color="b", label="blue: neg dis")
                plt.legend()
                plt.xlim(0, 1)
                plt.show()


            print("Epoch: {:>3} | Train Loss: {:+8.2f} | Val Loss: {:+8.3f} | Train Neck: {:+8.3f} | Val Neck: {:+8.3f} | Train Recon: {:+8.3f} | Val Recon: {:+8.3f}"
                  .format( i + 1,    avg_train_cost,         val_cost,        avg_train_neck_cost,      val_neck_cost,       avg_train_pred_cost,    val_pred_loss))
            print("---------- | Train AUC: {:+8.3f} | Val AUC: {:+8.3f} | TF Break: {:06.3f} | mean pos dist: {:06.3f} | mean neg dist {:06.3f} "
                  .format(avg_train_auc,val_auc,   true_false_break, a, -b))
            print()

    Ytest = np.reshape(Ytest, [-1,1])


    # Testing
    test_cost, test_neck_cost, pred_test = sess.run([loss, loss_neck_distance, neck_pred],  # {loss:test_cost, loss_neck_distance:test_neck_cost, neck_pred:pred_test}
                                                   feed_dict={X: Xtest, label: Ytest, training: False, pos_neg_break: true_false_break})
    test_auc = roc_auc_score(Ytest, pred_test)  # getting testing auc score
    print("Test Loss: {:02.5f} | Neck Loss: {:02.5f} | AUC: {:02.5f}".format(test_cost, test_neck_cost, test_auc))

training data size: (3000, 2118) | validate data size: (1000, 2118) | testing data size: (1000, 2118)


W0709 16:12:18.485413 14308 deprecation.py:323] From C:\Users\ziyangding\AppData\Local\Continuum\anaconda3\lib\site-packages\tensorflow\python\ops\metrics_impl.py:809: div (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.


AttributeError: module 'tensorflow._api.v1.train' has no attribute 'RMXPropOptimizer'

In [237]:
from keras.models import Model
from keras.layers import Input, Dense, Dropout, Subtract, Lambda
import keras.backend as K
from keras.losses import mean_squared_error


batch = 512

data = raw.sample(5000)

# Splitting dataframe into train, validation, and testing
dataY = data['label'].values
dataX = data.drop(columns = 'label').values


X, Xtest, Y, Ytest = train_test_split(dataX, dataY, test_size = 0.2, random_state = 42)
Xtrain, Xval, Ytrain, Yval = train_test_split(X, Y, test_size = 0.25, random_state = 42)


break_index = data.columns.values.tolist().index("item_rate") # first item index-1 is the break index
length_total = data.values.shape[1]
length_p = break_index # index of last user feature into length of the user feature
length_g = length_total-length_p-1


def pgSplit(data, idx):
    data_p = data[:, :idx]
    data_g = data[:, idx:]
    return data_p, data_g

Xtrain_p, Xtrain_g = pgSplit(Xtrain, break_index)
Xval_p, Xval_g = pgSplit(Xval, break_index)
Xtest_p, Xtest_g = pgSplit(Xtest, break_index)

a = 1
num_encode_1 = int(256 *a)
num_encode_2 = int(128 *a)
num_encode_3 = int(64 *a)
num_neck = 100
num_decode_1 = num_encode_3
num_decode_2 = num_encode_2
num_decode_3 = num_encode_1
num_output_to_p = length_p
num_output_to_g = length_g

threshold = 0.5 * math.sqrt(num_neck)


In [238]:
label = Input(shape=(1,))

## person autoencoder
main_p_input = Input(shape=(length_p,))
encode_p_1 = Dense(num_encode_1, activation='relu')(main_p_input)
encode_dropout_p_1 = Dropout(0.1)(encode_p_1)
encode_p_2 = Dense(num_encode_2, activation='relu')(encode_dropout_p_1)
encode_dropout_p_2 = Dropout(0.1)(encode_p_2)
encode_p_3 = Dense(num_encode_3, activation='relu')(encode_dropout_p_2)
encode_dropout_p_3 = Dropout(0.1)(encode_p_3)
encode_p_neck = Dense(num_neck, activation= 'sigmoid')(encode_dropout_p_3) ###
decode_p_1 = Dense(num_decode_1, activation='relu')(encode_p_neck)
decode_dropout_p_1 = Dropout(0.1)(decode_p_1)
decode_p_2 = Dense(num_decode_2, activation='relu')(decode_dropout_p_1)
decode_dropout_p_2 = Dropout(0.1)(decode_p_2)
decode_p_3 = Dense(num_decode_3, activation='relu')(decode_dropout_p_2)
decode_dropout_p_3 = Dropout(0.1)(decode_p_3)
output_p_out = Dense(num_output_to_p, activation= 'sigmoid', name = "p_out")(decode_dropout_p_3)

## goods autoencoder
main_g_input = Input(shape=(length_g,))
encode_g_1 = Dense(num_encode_1, activation='relu')(main_g_input)
encode_dropout_g_1 = Dropout(0.1)(encode_g_1)
encode_g_2 = Dense(num_encode_2, activation='relu')(encode_dropout_g_1)
encode_dropout_g_2 = Dropout(0.1)(encode_g_2)
encode_g_3 = Dense(num_encode_3, activation='relu')(encode_dropout_g_2)
encode_dropout_g_3 = Dropout(0.1)(encode_g_3)
encode_g_neck = Dense(num_neck, activation= 'sigmoid')(encode_dropout_g_3) ###
decode_g_1 = Dense(num_decode_1, activation='relu')(encode_g_neck)
decode_dropout_g_1 = Dropout(0.1)(decode_g_1)
decode_g_2 = Dense(num_decode_2, activation='relu')(decode_dropout_g_1)
decode_dropout_g_2 = Dropout(0.1)(decode_g_2)
decode_g_3 = Dense(num_decode_3, activation='relu')(decode_dropout_g_2)
decode_dropout_g_3 = Dropout(0.1)(decode_g_3)
output_g_out = Dense(num_output_to_g, activation= 'sigmoid', name = "g_out")(decode_dropout_g_3)

y_recon = tf.concat((output_p_out, output_g_out),1)
y_truth = tf.concat((main_p_input, main_g_input),1)

def AUC(y_true, label):
    auc = tf.metrics.auc(y_true,label)[1]
    K.get_session().run(tf.local_variables_initializer())
    return auc

def getOutput(X):
    distance = tf.norm(X, axis = 1)
    output = K.cast(tf.less_equal(distance, threshold), tf.float32)
    return output

distance = Subtract()([encode_p_neck, encode_g_neck])
output = Lambda(getOutput)(distance)


# Create a loss function that adds the MSE loss to the mean of all squared activations of a specific layer
def reconstructionLoss(y_recon, y_truth):
    return mean_squared_error(y_recon, y_truth)

def covarianceLoss(neck_p, neck_g):
    X = tf.concat((neck_p, neck_g), 0)
    n_rows = tf.cast(tf.shape(X)[0], tf.float32)
    X = X - (tf.reduce_mean(X, axis=0))
    cov = tf.matmul(X, X, transpose_a=True) / n_rows
    return tf.reduce_sum(tf.matrix_set_diag(cov, tf.zeros(num_neck, tf.float32)))

def distanceLoss(neck_p, neck_g, label):
    distance = tf.norm(neck_p - neck_g, axis = 1)
    sign = 2*label-1
    return tf.reduce_mean(tf.maximum(0.0, 0.6*threshold+tf.multiply(sign, distance-threshold)))

losses = {"reconstruction loss": reconstructionLoss
          "covariance loss": covarianceLoss
          "distance loss": distanceLoss}

weights = {"reconstruction loss": 1
          "covariance loss": 1
          "distance loss": 1}

In [240]:
model = Model(inputs= [main_p_input, main_g_input, label], outputs = [output,  ,y_recon])
model.compile(optimizer='rmsprop', loss=losses, loss_weights=weights , metrics=[AUC])
model.fit([Xtrain_p, Xtrain_g, Ytrain], Ytrain, validation_data=([Xval_p, Xval_g, Yval], Yval), epochs=20, batch_size=batch)

NameError: name 'weights' is not defined